In [11]:
from data_prep import SystemIdentDataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

train_dataset = SystemIdentDataset(train=True)
val_dataset = SystemIdentDataset(train=False)
train_dloader = DataLoader(train_dataset, batch_size=32,shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
def evaluate(model, loss_function, val_loader):
    running_loss = 0.0
    p_error = 0.0
    batch_count = 0
    for _, example in enumerate(tqdm(val_loader), 0):
        inputs,label = example

        with torch.no_grad():
            outputs = model(inputs)
        
        loss = loss_fn(outputs, label)
        p_error += (torch.sum(torch.abs(outputs-label)/((label+outputs)/2))/(4*val_dloader.batch_size)).item()
        running_loss += loss.item()
        batch_count += 1
    p_error /= batch_count
    p_error *= 100
    return running_loss, p_error

def train(model, num_epochs, loss_function, optimizer, train_loader, val_loader):
    val_loss, p_error= evaluate(model, loss_function, val_loader)
    print(f"Initial validation loss: {val_loss} and percent error: {p_error}")
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1}:")
        for _, example in enumerate(tqdm(train_loader), 0):
            inputs,label = example
            
            outputs = model(inputs)

            loss = loss_fn(outputs, label)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()
        val_loss, p_error = evaluate(model, loss_function, val_loader)
        print(f"validation loss: {val_loss}, percent error: {p_error}")

def transform_state(s):
    max_v = torch.sqrt()



In [12]:

# Define the model
model = torch.nn.Sequential(
    torch.nn.Linear(4, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 64, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(64, 32, dtype=torch.float64),
    torch.nn.Tanh(),
    torch.nn.Linear(32, 4, dtype=torch.float64),
)

# Define the loss function
loss_fn = torch.nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())
train(model=model, num_epochs=20, loss_function=loss_fn, optimizer=optimizer, train_loader=train_dloader, val_loader=val_dloader)



100%|██████████| 21875/21875 [00:13<00:00, 1584.52it/s]


Initial validation loss: 3264.289464830364 and percent error: 2.849474678099448
Epoch 1:


 17%|█▋        | 14489/87500 [00:19<01:38, 741.33it/s]


KeyboardInterrupt: 